### Entregable 1

In [ ]:
import requests
import json
response_API = requests.get("https://chilealerta.com/api/query/?user=demo&select=ultimos_sismos&limit=100&country=Chile")
print(response_API.status_code)

In [ ]:
#JSON a diccionario
ultimos_sismos = json.loads(response_API.text)
ultimos_sismos = ultimos_sismos['ultimos_sismos_Chile']
ultimos_sismos[0:5]

### Entregable 2

In [ ]:
#!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os 

url= os.getenv('URL')
data_base= os.getenv('DATA_BASE')
user= os.getenv('USER')
pwd= os.getenv('PWD')
port = os.getenv('PORT')

In [ ]:
# Creando la conexión a Redsshift
import psycopg2
try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port=port
    )
    print("¡Conexión existosa!")
    
except Exception as e:
    print("¡Conexión invalida!", e)

In [ ]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS sfparevalo_coderhouse.earthquake (
            id INTEGER primary key,
            state_earthquake INTEGER,
            utc_time TIMESTAMP,
            reference VARCHAR(255),
            magnitude DECIMAL(5, 2),
            scale_earthquake VARCHAR(5),
            latitude DECIMAL(8, 4),
            longitude DECIMAL(8, 4),
            depth_earthquake DECIMAL(8, 2)
        ) DISTSTYLE EVEN SORTKEY (utc_time);
    """)
    conn.commit()

In [ ]:
import pandas as pd
#Consultar la ultima fecha del ultimo sismo registrado en la base de datos.
with conn.cursor() as cur:
    cur.execute('''
        SELECT utc_time
        FROM earthquake
        ORDER BY utc_time DESC
        LIMIT 1;
        ''')
    ultima_fecha = cur.fetchone()
    if ultima_fecha is None:
        ultima_fecha = pd.Timestamp.min #Fecha mas baja tipo Timestamp.
    else:
        ultima_fecha = ultima_fecha[0]
        


In [ ]:
print(ultima_fecha)

In [1]:
"""
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
    cur.execute("Truncate table earthquake")
    count = cur.rowcount
"""

'\n#Vaciar la tabla para evitar duplicados o inconsistencias\nwith conn.cursor() as cur:\n    cur.execute("Truncate table earthquake")\n    count = cur.rowcount\n'

In [ ]:
df = pd.DataFrame(ultimos_sismos)

In [ ]:
#Eliminar elementos duplicados
df.drop_duplicates(subset=['id'], inplace=True)

In [ ]:
#Transformar el campo fecha a tipo fecha.
df['utc_time']= pd.to_datetime(df['utc_time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df.dtypes

In [ ]:
# Verificar si hay datos nulos en cada columna
df.isnull().sum()

In [ ]:
# Eliminar registros que posean valores nulos 
df = df.dropna(how='any',axis=0)

In [ ]:
#Considerar datos que se van a guardar en la base de datos.
df = df[['id', 'state', 'utc_time', 'reference', 'magnitude', 'scale', 'latitude', 'longitude', 'depth']]
df.head()

In [ ]:
# Considerar solo los nuevos sismos desde la ultima fecha guardada en la base de datos
df = df[df['utc_time'] > ultima_fecha]
df

In [ ]:
#Filtrar para los sismos con una magnitud mayor a 5mb escala richter.
df_magnitude = df[df['magnitude'] > 5]
df_magnitude

In [ ]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, body, sender, recipients, password):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender, password)
        smtp_server.sendmail(sender, recipients, msg.as_string())
    print("Message sent!")


In [ ]:
#Si existen sismos con una magnitud mayor a 5 grados en al escala de richter, se envia el correo.

if(df_magnitude.shape[0] > 0):
    sismos_string = ""
    for index, row in df_magnitude.iterrows():  
        sismos_string += f"Fecha: {row['utc_time']} UTC, Lugar: {row['reference']}, Magnitud: {row['magnitude']} grados escala richter\n"
    subject = "Ultimos sismos - Magnitudes mayor a 5, escala richter"
    body = sismos_string
    sender = "pitersito5647@gmail.com"
    recipients = ["pitersito5647@gmail.com"]
    password = "jjsu ikuc ghcl mpgq"
    send_email(subject, body, sender, recipients, password)
else:
    print("No hubieron sismos mayores a 5 grados en la escala de richter")
    

In [ ]:
from psycopg2.extras import execute_values

tuple_rows = [tuple(row) for row in df.values]
with conn.cursor() as cur:
    try:
        execute_values(
            cur,
            '''
            INSERT INTO earthquake (id, state_earthquake, utc_time, reference, magnitude, scale_earthquake, latitude, longitude, depth_earthquake) 
            VALUES %s 
            ''',
            tuple_rows
        )
        conn.commit()
        print("¡Datos ingresados satisfactoriamente!")
    except psycopg2.Error as err:
        print(err)


In [ ]:
#Cerrar conexión
cur.close()
conn.close()